## Model Deployment Testing Notebook

This notebook demonstrates:
 - Testing a local API endpoint for inference.
 - Testing the API hosted in a Docker container.
 - Sending parallel requests to evaluate the API's performance under load.

In [37]:
import requests
import json

from concurrent.futures import ThreadPoolExecutor
import asyncio
import aiohttp
import nest_asyncio

**1. Test the local API**

In [38]:
# Define the local API URL and input data
local_url = 'http://127.0.0.1:8000/infer'
test_data = {'input': 'I am satisfied with the new shoes I bought from the store.'}

# Send a POST request to the local API
%time response = requests.post(local_url, data=json.dumps(test_data), headers={'Content-Type': 'application/json'})

# Print the complete response
print("Full Response:", response.json())


CPU times: user 0 ns, sys: 5.37 ms, total: 5.37 ms
Wall time: 61.7 ms
Full Response: {'prediction': [{'label': 'positive', 'score': 0.9680546522140503}]}


In [39]:
# Extract specific fields from the response
label = json.loads(response.text)['prediction'][0]['label']
score = json.loads(response.text)['prediction'][0]['score']
print(f"Predicted Label: {label}")
print(f"Confidence Score: {score}")

Predicted Label: positive
Confidence Score: 0.9680546522140503


**2. Test the local dockerized api**

In [40]:
docker_url = 'http://0.0.0.0:8000/infer'
docker_test_data = {'input': 'I am not satisfied with the new shoes I bought from the store.'}

# Send a POST request to the Dockerized API
%time response = requests.post(docker_url, data=json.dumps(docker_test_data), headers={'Content-Type': 'application/json'})

# Print the complete response
print("Full Response:", response.json())

CPU times: user 3.77 ms, sys: 1.16 ms, total: 4.92 ms
Wall time: 36.9 ms
Full Response: {'prediction': [{'label': 'negative', 'score': 0.8943052291870117}]}



**3. Parallel Requests, Dockerized API**

In [41]:
# Define a function to make a single POST request
def make_request():
    response = requests.post(docker_url, json=payload)
    print(f"Status Code: {response.status_code}, Response: {response.json()}")

# Define the payload and send 10 parallel requests
payload = {"input": "I love these shoes!"}
print("Sending 10 parallel requests to the Dockerized API...\n")

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(make_request) for _ in range(10)]




Sending 10 parallel requests to the Dockerized API...



Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}


In [42]:
nest_asyncio.apply()  # Allow running nested event loops in Jupyter Notebook

async def make_request(session):
    async with session.post(docker_url, json=payload) as response:
        result = await response.json()
        print(f"Status Code: {response.status}, Response: {result}")

async def send_parallel_requests():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for _ in range(10):
            task = asyncio.create_task(make_request(session))
            tasks.append(task)
        await asyncio.gather(*tasks)

# Define the payload and send 10 parallel requests
payload = {"input": "I love these shoes!"}
print("Sending 10 parallel requests to the Dockerized API...\n")

# Run the event loop to send parallel requests
loop = asyncio.get_event_loop()
loop.run_until_complete(send_parallel_requests())

Sending 10 parallel requests to the Dockerized API...

Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score': 0.9823362231254578}]}
Status Code: 200, Response: {'prediction': [{'label': 'positive', 'score